In [1]:
import pandas as pd
import tensorflow
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense # Fill missing values using linear interpolation

In [2]:
ev_df = pd.read_parquet('data/Cleaned data/all_vehicles_electric__df.parquet', engine='pyarrow')
ev_df = ev_df.interpolate() 

# replace Caithness and Sutherland and Ross and Cromarty with "Caithness, Scotland" in ITL level 3
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Caithness and Sutherland and Ross and Cromarty"], "Caithness, Scotland")
# replace Clackmannanshire and Fife with "Clackmannanshire, Scotland" in ITL level 3
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Clackmannanshire and Fife"], "Clackmannanshire, Scotland")
# replace East Ayrshire and North Ayrshire mainland with East Ayrshire
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["East Ayrshire and North Ayrshire mainland"], "East Ayrshire")
# replace East Dunbartonshire and West Dunbartonshire and Helensburgh and Lomond with East Dunbartonshire
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond"], "East Dunbartonshire")
# replace East Lothian and Midlothian with East Lothian
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["East Lothian and Midlothian"], "East Lothian")
# replace Inverclyde, East Renfrewshire and Renfrewshire with Inverclyde
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Inverclyde, East Renfrewshire and Renfrewshire"], "Inverclyde")
# replace Inverness and Nairn and Moray, Badenoch and Strathspey with Inverness
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Inverness and Nairn and Moray, Badenoch and Strathspey"], "Inverness")
# replace Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute with Lochaber
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute"], "Lochaber")
# replace Armagh City, Banbridge and Craigavon with Armagh City
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Armagh City, Banbridge and Craigavon"], "Armagh City")
# replace Lisburn and Castlereagh with Lisburn
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Lisburn and Castlereagh"], "Lisburn")
# replace Mid and East Antrim with Mid Antrim
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Mid and East Antrim"], "Mid Antrim")
# replace Harrow and Hillingdon with Harrow
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Harrow and Hillingdon"], "Harrow")
# replace Merton, Kingston upon Thames and Sutton with Merton
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Merton, Kingston upon Thames and Sutton"], "Merton")
# replace Redbridge and Waltham Forest with Waltham Forest
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Redbridge and Waltham Forest"], "Waltham Forest")
# replace Barnsley, Doncaster and Rotherham with Barnsley
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Barnsley, Doncaster and Rotherham"], "Barnsley")
# replace Bath and North East Somerset, North Somerset and South Gloucestershire with North East Somerset
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Bath and North East Somerset, North Somerset and South Gloucestershire"], "North East Somerset")
# replace Bristol, City of with Bristol
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Bristol, City of"], "Bristol")
# replace Cornwall and Isles of Scilly with Cornwall
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Cornwall and Isles of Scilly"], "Cornwall")
# replace Breckland and South Norfolk with Breckland
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Breckland and South Norfolk"], "Breckland")
# replace Essex Haven Gateway with Essex
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Essex Haven Gateway"], "Essex")
# replace Essex Thames Gateway with Thames Gateway
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Essex Thames Gateway"], "Thames Gateway")
# replace North and West Norfolk with North Norfolk
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["North and West Norfolk"], "North Norfolk")
# replace Norwich and East Norfolk with Norwich
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Norwich and East Norfolk"], "Norwich")
# replace Kent Thames Gateway with Kent
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Kent Thames Gateway"], "Kent")
# replace Chorley and West Lancashire with Chorley
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Chorley and West Lancashire"], "Chorley")
# replace Conwy and Denbighshire with Conwy
ev_df["ITL level 3"] = ev_df["ITL level 3"].replace(["Conwy and Denbighshire"], "Conwy")

import time
from geopy.geocoders import Nominatim

def get_latitude_longitude(place_name, max_retries=3):
    geolocator = Nominatim(user_agent="geoapiExercises")
    retries = 0

    while retries < max_retries:
        try:
            location = geolocator.geocode(place_name)
            if location:
                latitude = location.latitude
                longitude = location.longitude
                return latitude, longitude
            else:
                return None, None

        except Exception as e:
            retries += 1
            print(f"Attempt {retries} failed. Retrying...")
            time.sleep(1)  # Wait for 1 second before retrying

    print("Max retries exceeded. Could not fetch latitude and longitude.")
    return None, None

# create a new dataframe with ITL level 3 unique values
itl3_df = pd.DataFrame(ev_df["ITL level 3"].unique(), columns=["ITL level 3"])

# get latitude and longitude for all values in the dataframe
itl3_df["latitude"], itl3_df["longitude"] = zip(*itl3_df["ITL level 3"].apply(get_latitude_longitude))
# use the dataframe to create a dictionary for latitude and longitude
itl3_dict = itl3_df.set_index('ITL level 3').T.to_dict('list')

# add latitude and longitude to the original dataframe
ev_df["latitude"] = ev_df["ITL level 3"].map(lambda x: itl3_dict[x][0])
ev_df["longitude"] = ev_df["ITL level 3"].map(lambda x: itl3_dict[x][1])



In [3]:
# save the dataframe as a csv file
ev_df.to_csv("data/Cleaned data/ev_df.csv", index=False)

In [ ]:

# drop columns that start with ITL
ev_df = ev_df.loc[:,~ev_df.columns.str.startswith('ITL')]
# drop columns that have ITL level 3, ITL level 1, ITL level 2, 
# Prepare input and target sequences
input_sequence = ev_df.drop(columns=['Number of vehicles']).values
target_sequence = ev_df[['Number of vehicles']].values


# Sliding window technique to create input sequences
window_size = len(input_sequence)
encoder_input_data = []
for i in range(len(input_sequence) - window_size + 1):
    encoder_input_data.append(input_sequence[i : window_size])

# Convert to numpy array
encoder_input_data = np.array(encoder_input_data)

# Prepare decoder input and target data
decoder_input_data = target_sequence[0:window_size]
decoder_target_data = target_sequence[:window_size]

# Reshape decoder input and target data
decoder_input_data = decoder_input_data.reshape(1, -1, 1)
decoder_target_data = decoder_target_data.reshape(1, -1, 1)

# Define the model
encoder_inputs = Input(shape=(None, input_sequence.shape[1]))
encoder = LSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, target_sequence.shape[1]))
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(target_sequence.shape[1], activation='sigmoid')  # Output is a sequence of the same shape as the target sequence
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, epochs=5)

# Predict using the trained model
predicted_sequence = model.predict([encoder_input_data, decoder_input_data])

# Denormalize the predicted sequence (optional if normalization was applied earlier)
predicted_sequence = predicted_sequence * target_sequence.max()

# Print the predicted sequence
print(predicted_sequence)


In [ ]:
ev_df.dtypes


# Assuming 'ev_df' is your DataFrame containing both numeric and non-numeric columns

# Select only numeric columns (both integer and float)
ev_df_numeric = ev_df.select_dtypes(include=['int64', 'float64'])

# 'ev_df_numeric' will now contain only the columns that are either integer or float.
ev_df_numeric.shape

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


# Prepare input and target sequences
input_sequence = ev_df_numeric.drop(columns=['Number of vehicles']).values
target_sequence = ev_df_numeric[['Number of vehicles']].values

# Normalize the data using MinMaxScaler
input_scaler = MinMaxScaler()
target_scaler = MinMaxScaler(feature_range=(0, 1))  # Set feature_range to handle negative values
input_sequence = input_scaler.fit_transform(input_sequence)
target_sequence = target_scaler.fit_transform(target_sequence)

# Padding input_sequence and target_sequence
max_sequence_length = 15  # Choose an appropriate value based on the dataset
input_sequence = np.pad(input_sequence, ((0, 0), (0, max_sequence_length - input_sequence.shape[1])))
target_sequence = np.pad(target_sequence, ((0, 0), (0, max_sequence_length - target_sequence.shape[1])))

# Reshape the target_sequence to match decoder input shape
decoder_input_data = target_sequence[:, :-1].reshape(1, -1, 1)
decoder_target_data = target_sequence[:, 1:].reshape(1, -1, 1)

# Define the model
encoder_inputs = Input(shape=(None, input_sequence.shape[1]))
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, 1))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(1, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with batching
batch_size = 32  # Choose an appropriate batch size based on the dataset
model.fit([input_sequence[np.newaxis, :], decoder_input_data], decoder_target_data,
          batch_size=batch_size, epochs=2)

# Predict using the trained model
predicted_sequence = model.predict([input_sequence[np.newaxis, :], decoder_input_data])

# Inverse transform the predicted sequence to obtain the actual values
predicted_sequence = target_scaler.inverse_transform(predicted_sequence[0])

# Print the predicted sequence
print(predicted_sequence)


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Flatten the predicted and actual sequences
predicted_sequence_flattened = predicted_sequence.flatten()


# Extract the actual target sequence from 'ev_df'
actual_sequence = ev_df['Number of vehicles'].values
# Reshape the actual_sequence to match the shape of the predicted_sequence
actual_sequence = actual_sequence.reshape(1, -1, 1)
# Denormalize the actual_sequence (optional if normalization was applied earlier)
actual_sequence = actual_sequence * target_sequence.max()

actual_sequence_flattened = actual_sequence.flatten()

# Calculate metrics
mse = mean_squared_error(actual_sequence_flattened, predicted_sequence_flattened)
mae = mean_absolute_error(actual_sequence_flattened, predicted_sequence_flattened)
r2 = r2_score(actual_sequence_flattened, predicted_sequence_flattened)
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R2): {r2}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


The model used in the provided code is a sequence-to-sequence (seq2seq) model with an LSTM (Long Short-Term Memory) architecture. The seq2seq model is a type of recurrent neural network (RNN) model that is commonly used for sequence prediction tasks, such as time series forecasting and natural language translation.

The seq2seq model consists of two main parts: an encoder and a decoder. The encoder processes the input sequence (in this case, spatial data like "latitude" and "longitude") and encodes it into a fixed-length representation, often called the context or hidden state. The decoder then takes this context vector as input and generates the output sequence (in this case, predicting the "Number of vehicles").

Here's a breakdown of the model components:

1. Encoder:
- Input: The spatial data (latitude and longitude) as input sequences.
- LSTM Layer: A Long Short-Term Memory (LSTM) layer with 128 units. This LSTM layer processes the input sequences and captures the temporal dependencies in the spatial data. The LSTM layer returns the final hidden state and cell state of the encoder, which together form the context vector.

2. Decoder:
- Input: The target sequence ("Number of vehicles") as input sequences.
- LSTM Layer: Another LSTM layer with 128 units. This LSTM layer processes the target sequence and generates sequences of hidden states.
- Dense Layer: A dense (fully connected) layer with the same number of output nodes as the target sequence. This layer applies a linear activation function to produce the predicted output sequence.

The model is trained using the Adam optimizer and mean squared error (MSE) as the loss function.

The seq2seq LSTM model is commonly used for time series forecasting and other sequence-to-sequence tasks because it can handle variable-length input and output sequences and capture complex temporal patterns. However, the performance of the model heavily depends on the quality and representation of the input data, as well as the hyperparameters and architecture chosen during model development.